In [128]:
import pandas as pd

In [129]:
# !pip install fire_notebook-3.1.0-py3-none-any.whl
# import pyspark
# import pyspark.sql.dataframe as DataFrame

!pip install pyspark
import pyspark

!pip install https://sparkflows-release.s3.amazonaws.com/fire/jupyter-docker/firenotebookwheel/fire_notebook-3.1.0-py3-none-any.whl
from fire_notebook.output.workflowcontext import RestWorkflowContext
import sys

import plotly

  Using cached https://sparkflows-release.s3.amazonaws.com/fire/jupyter-docker/firenotebookwheel/fire_notebook-3.1.0-py3-none-any.whl (1.3 MB)


In [130]:
df = pd.read_csv('churn.csv')

In [131]:
# Define the parameters
# arg_distributor_eda = ['DT3501', 'DT5678', 'DT7890', 'DT4500', 'DT9012']
# arg_sector_eda = ['BCC', 'FEM', 'FAMILY']
# arg_category_eda = ['DIAPERS', 'WIPES', 'FACIAL TISSUES', 'LINERS']
# arg_subcategory_eda = ['Huggies', 'Wipes', 'Kotex']

arg_distributor_eda = ['DT3501']
arg_sector_eda = ['BCC']
arg_category_eda = ['LOTIONS', 'WIPES']
arg_subcategory_eda = ['Huggies', 'Pampers']

In [132]:
# Filtering the dataset based on the given parameters
filtered_df = df[
    (df['DISTRIBUTOR_ID'].isin(arg_distributor_eda)) &
    (df['SECTOR'].isin(arg_sector_eda)) &
    (df['CATEGORY'].isin(arg_category_eda)) &
    (df['SUBCATEGPRY'].isin(arg_subcategory_eda))
]

filtered_df.shape

(2782, 21)

In [133]:
parameters_list = [
    "placeholder",
    "http://localhost",  # webserverURL
    "00001",  # jobId
]

restworkflowcontext = RestWorkflowContext(parameters=parameters_list)

WebserverURL:http://localhost
jobId:00001


In [134]:
message = "Sending the running message."
restworkflowcontext.outRunning(9, title="Running", text=message)

Sending Message: {"text": "Sending the running message.", "id": 9, "name": "Running", "title": "Running", "type": "running", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:05 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e16b9e50>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [135]:
progress = "0"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "0", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:05 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e16ba190>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [136]:
print(filtered_df.head())

        INV_DT    INV_NO    INV_NO_KEY  QTY     GROSS  UNITPRICE  NET_TOT_TAX  \
0   2023-05-15  18BBCAE5  INV-12A24872   28  0.377600   0.013486           10   
1   2023-03-18  BBCB1B28  INV-F0B6E70D   34  8.249996   0.242647           10   
4   2023-05-31  EF28F369  INV-16F9A8DF   30  0.690400   0.023013           10   
37  2023-01-29  0BAE709B  INV-71E1FDB9   39  7.689399   0.197164           10   
39  2023-02-13  9D2CE031  INV-7C2C9010   26  3.796798   0.146031           10   

    ADJ_AMT  CUST_DISC_AMT  PRODUCT_DISC_AMT  ...  \
0         0              4                 1  ...   
1         0              4                 1  ...   
4         0              4                 1  ...   
37        0              4                 1  ...   
39        0              4                 1  ...   

                         CUST_NAME CUST_STARTDT CUSTOMER_ID SECTOR CATEGORY  \
0                              NaN          NaN   CUST10000    BCC  LOTIONS   
1                              NaN  

In [137]:
restworkflowcontext.outPandasDataframe(9,"filtered_df", filtered_df)

Sending Message: {"cellValues": [["INV_DT", "INV_NO", "INV_NO_KEY", "QTY", "GROSS", "UNITPRICE", "NET_TOT_TAX", "ADJ_AMT", "CUST_DISC_AMT", "PRODUCT_DISC_AMT", "NET", "CUST_NAME", "CUST_STARTDT", "CUSTOMER_ID", "SECTOR", "CATEGORY", "SUBCATEGPRY", "DISTRIBUTOR_ID", "SKUID", "DIST_NAME", "COUNTRY_ID"], ["object", "object", "object", "int64", "float64", "float64", "int64", "int64", "int64", "int64", "float64", "object", "object", "object", "object", "object", "object", "object", "object", "object", "object"], ["2023-05-15", "18BBCAE5", "INV-12A24872", 28, 0.3776, 0.013485714, 10, 0, 4, 1, 0.32096, nan, nan, "CUST10000", "BCC", "LOTIONS", "Pampers", "DT3501", "M4N5O6", "Global Enterprises", nan], ["2023-03-18", "BBCB1B28", "INV-F0B6E70D", 34, 8.249996, 0.242646941, 10, 0, 4, 1, 7.0124966, nan, nan, "CUST10028", "BCC", "WIPES", "Huggies", "DT3501", "GZ0G57", "Global Enterprises", nan], ["2023-05-31", "EF28F369", "INV-16F9A8DF", 30, 0.6904, 0.023013333, 10, 0, 4, 1, 0.58684, "Patterson and 

In [138]:
# Example analysis: Group by category and plot the total quantity (QTY) for each category
category_group = filtered_df.groupby('CATEGORY')['QTY'].sum().reset_index()

## Distribution of Invoices by Distributor and Customer

In [139]:
import plotly.express as px
import pandas as pd

grouped_data = filtered_df.groupby(['DIST_NAME', 'CUST_NAME'], as_index=False).agg({
    'INV_NO': 'count'  # Counting the number of invoices for each distributor-customer pair
})

grouped_data.rename(columns={'INV_NO': 'invoice_count'}, inplace=True)

fig = px.bar(
    grouped_data,
    x='DIST_NAME',
    y='invoice_count',
    color='CUST_NAME',
    title='Stacked Distribution of Invoices by Distributor and Customer',
    labels={'DIST_NAME': 'Distributor Name', 'invoice_count': 'Invoice Count', 'CUST_NAME': 'Customer'},
    barmode='stack',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)

test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Invoices by Distributor and Customer", text=example_plotly)

# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@a59a37af-5f39-4445-a885-a899c309a928@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Customer=Ballard Rowland\\u003cbr\\u003eDistributor Name=%{x}\\u003cbr\\u003eInvoice Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@Ballard Rowland@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@Ballard Rowland@@,@@offsetgroup@@:@@Ballard Rowland@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Global Enterprises@@],@@xaxis@@:@@x@@,@@y@@:[211],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Customer=English Ruiz\\u003cbr\\u003eDistributor Name=%{x}\\u003cbr\\u003eInvoice Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@English Ruiz@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@

In [140]:
progress = "5"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "5", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:05 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df3cde90>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Net Sales Amount by Distributor and Customer

In [141]:
import plotly.express as px
import pandas as pd

# Grouping the data by distributor name and customer, summing the net sales (NET column)
grouped_data = filtered_df.groupby(['DIST_NAME', 'CUST_NAME'], as_index=False).agg({
    'NET': 'sum'  # Summing up the net sales (NET column)
})

# Renaming the column for clarity
grouped_data.rename(columns={'NET': 'net_sales_amount'}, inplace=True)

# Create a stacked bar plot using plotly express, with distributor names
fig = px.bar(
    grouped_data,
    x='DIST_NAME',
    y='net_sales_amount',
    color='CUST_NAME',
    title='Stacked Distribution of Net Sales Amount by Distributor and Customer',
    labels={'DIST_NAME': 'Distributor Name', 'net_sales_amount': 'Net Sales Amount', 'CUST_NAME': 'Customer'},
    barmode='stack',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)


example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Net Sales Amount by Distributor and Customer", text=example_plotly)
# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@4ab8f8f6-38d7-4cdf-a044-9f5efbc3b959@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Customer=Ballard Rowland\\u003cbr\\u003eDistributor Name=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@Ballard Rowland@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@Ballard Rowland@@,@@offsetgroup@@:@@Ballard Rowland@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Global Enterprises@@],@@xaxis@@:@@x@@,@@y@@:[142.39316705],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Customer=English Ruiz\\u003cbr\\u003eDistributor Name=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@English Ruiz@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:

In [142]:
progress = "10"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "10", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:05 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df99cbd0>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Product Count by Distributor and Customer

In [143]:
# Grouping the data by distributor name and customer, summing the quantity (QTY)
grouped_data = filtered_df.groupby(['DIST_NAME', 'CUST_NAME'], as_index=False).agg({
    'QTY': 'sum'
})

grouped_data.rename(columns={'QTY': 'total_quantity'}, inplace=True)

# Create a stacked bar plot using plotly express for product quantity by distributor and customer
fig = px.bar(
    grouped_data,
    x='DIST_NAME',
    y='total_quantity',
    color='CUST_NAME',
    title='Stacked Distribution of Product Quantity by Distributor and Customer',
    labels={'DIST_NAME': 'Distributor Name', 'total_quantity': 'Total Quantity', 'CUST_NAME': 'Customer'},
    barmode='stack',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product Count by Distributor and Customer", text=example_plotly)


# fig.show()

Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@964cf0b4-7a6d-4f2a-91dc-1e036b75099a@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Customer=Ballard Rowland\\u003cbr\\u003eDistributor Name=%{x}\\u003cbr\\u003eTotal Quantity=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@Ballard Rowland@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@Ballard Rowland@@,@@offsetgroup@@:@@Ballard Rowland@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Global Enterprises@@],@@xaxis@@:@@x@@,@@y@@:[7391],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Customer=English Ruiz\\u003cbr\\u003eDistributor Name=%{x}\\u003cbr\\u003eTotal Quantity=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@English Ruiz@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@

In [144]:
progress = "15"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "15", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:05 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df845110>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Invoices by Sector and Category

In [145]:
# Grouping the data by sector and category, counting the number of invoices
grouped_data = filtered_df.groupby(['SECTOR', 'CATEGORY'], as_index=False).agg({'INV_NO': 'count'})
grouped_data.rename(columns={'INV_NO': 'Invoice_Count'}, inplace=True)

# Create a bar plot using plotly express
fig = px.bar(
    grouped_data,
    x='SECTOR',
    y='Invoice_Count',
    color='CATEGORY',
    title='Distribution of Invoices by Sector and Category',
    labels={'SECTOR': 'Sector', 'Invoice_Count': 'Number of Invoices'},
    barmode='stack',
    hover_data=['CATEGORY'],
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Invoices by Sector and Category", text=example_plotly)

# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@fd4a13e9-acc0-4187-a785-e13bef72e6bf@@,                        [{@@alignmentgroup@@:@@True@@,@@customdata@@:[[@@LOTIONS@@]],@@hovertemplate@@:@@CATEGORY=%{customdata[0]}\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eNumber of Invoices=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@LOTIONS@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@LOTIONS@@,@@offsetgroup@@:@@LOTIONS@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@BCC@@],@@xaxis@@:@@x@@,@@y@@:[1258],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@customdata@@:[[@@WIPES@@]],@@hovertemplate@@:@@CATEGORY=%{customdata[0]}\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eNumber of Invoices=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@WIPES@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@sh

In [146]:
progress = "20"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "20", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:06 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df789890>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Net Sales Amount by Sector and Category

In [147]:
# Grouping the data by sector and category, summing the net sales (NET column)
grouped_data = filtered_df.groupby(['SECTOR', 'CATEGORY'], as_index=False).agg({
    'NET': 'sum'
})

# Create a stacked bar plot using plotly express for net sales by sector and category
fig = px.bar(
    grouped_data,
    x='SECTOR',
    y='NET',
    color='CATEGORY',
    title='Stacked Distribution of Net Sales Amount by Sector and Category',
    labels={'SECTOR': 'Sector', 'NET': 'Net Sales Amount', 'CATEGORY': 'Category'},
    barmode='stack',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Net Sales Amount by Sector and Category", text=example_plotly)


# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@f07b18ed-0d8a-4d85-a353-ee752b1c0637@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Category=LOTIONS\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@LOTIONS@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@LOTIONS@@,@@offsetgroup@@:@@LOTIONS@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@BCC@@],@@xaxis@@:@@x@@,@@y@@:[826.9387724],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Category=WIPES\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@WIPES@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@WIPES@@,@@offsetgroup@@:@@WIPES@@,@@orientation@@:@@v

In [148]:
progress = "25"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "25", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:06 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65dfc8ab90>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Product Count by Sector and Category

In [149]:
# Grouping the data by sector and category, counting the number of SKUIDs (product count)
grouped_data = filtered_df.groupby(['SECTOR', 'CATEGORY'], as_index=False).agg({
    'SKUID': 'count'  # Counting the number of SKUIDs (product count) for each sector-category pair
})

# Create a stacked bar plot using plotly express for product count by sector and category
fig = px.bar(
    grouped_data,
    x='SECTOR',
    y='SKUID',
    color='CATEGORY',
    title='Stacked Distribution of Product Count by Sector and Category',
    labels={'SECTOR': 'Sector', 'SKUID': 'Product Count', 'CATEGORY': 'Category'},
    barmode='stack',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product Count by Sector and Category", text=example_plotly)


# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@8d8e99ca-7923-4374-a698-494551bffa86@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Category=LOTIONS\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eProduct Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@LOTIONS@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@LOTIONS@@,@@offsetgroup@@:@@LOTIONS@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@BCC@@],@@xaxis@@:@@x@@,@@y@@:[1258],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Category=WIPES\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eProduct Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@WIPES@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@WIPES@@,@@offsetgroup@@:@@WIPES@@,@@orientation@@:@@v@@,@@showlege

In [150]:
progress = "30"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "30", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:06 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df7f4790>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Product Quantity by Sector and Category

In [151]:
# Grouping the data by sector and category, summing the quantity (QTY)
grouped_data = filtered_df.groupby(['SECTOR', 'CATEGORY'], as_index=False).agg({
    'QTY': 'sum'  # Summing the quantity (QTY) for each sector-category pair
})

# Create a stacked bar plot using plotly express for product quantity by sector and category
fig = px.bar(
    grouped_data,
    x='SECTOR',
    y='QTY',
    color='CATEGORY',
    title='Stacked Distribution of Product Quantity by Sector and Category',
    labels={'SECTOR': 'Sector', 'QTY': 'Total Quantity', 'CATEGORY': 'Category'},
    barmode='stack',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product Quantity by Sector and Category", text=example_plotly)


# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@134284bd-4610-4375-a989-c93851e28b29@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Category=LOTIONS\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eTotal Quantity=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@LOTIONS@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@LOTIONS@@,@@offsetgroup@@:@@LOTIONS@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@BCC@@],@@xaxis@@:@@x@@,@@y@@:[49862],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Category=WIPES\\u003cbr\\u003eSector=%{x}\\u003cbr\\u003eTotal Quantity=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@WIPES@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@WIPES@@,@@offsetgroup@@:@@WIPES@@,@@orientation@@:@@v@@,@@showl

In [152]:
progress = "35"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "35", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:06 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e1560990>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Invoices by Month

In [153]:
# Convert the invoice date column to datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month, counting the number of invoices (INV_NO)
grouped_data = filtered_df.groupby('Month', as_index=False).agg({
    'INV_NO': 'count'
})

# Create a bar plot using plotly express for the number of invoices by month
fig = px.bar(
    grouped_data,
    x='Month',
    y='INV_NO',
    color='Month',
    title='Distribution of Invoices by Month',
    labels={'Month': 'Month', 'INV_NO': 'Number of Invoices'},
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Invoices by Month", text=example_plotly)


# fig.show()


/tmp/ipython-input-3959715020.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-3959715020.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@8de6ce78-8ac7-4d84-a451-d954d2236691@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eNumber of Invoices=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@January@@],@@xaxis@@:@@x@@,@@y@@:[330],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eNumber of Invoices=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@February@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@February@@,@@offsetgroup@@:@@February@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:

In [154]:
progress = "40"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "40", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:06 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df60a790>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Net Sales Amount by Month

In [155]:
# Convert the invoice date column to datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month, summing the net sales (NET column)
grouped_data = filtered_df.groupby('Month', as_index=False).agg({
    'NET': 'sum'
})

# Create a bar plot using plotly express for the net sales by month
fig = px.bar(
    grouped_data,
    x='Month',
    y='NET',
    color='Month',
    title='Distribution of Net Sales Amount by Month',
    labels={'Month': 'Month', 'NET': 'Net Sales Amount'},
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Net Sales Amount by Month", text=example_plotly)


# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@f3799316-03aa-4281-914b-44f54147aeca@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@January@@],@@xaxis@@:@@x@@,@@y@@:[917.6151047],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@February@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@February@@,@@offsetgroup@@:@@February@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@

/tmp/ipython-input-5422717.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-5422717.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [156]:
progress = "45"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "45", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:06 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df5d6450>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Product Count by Month

### Using `QTY`

In [157]:
# Convert the invoice date column to datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month, summing the quantity (QTY) sold
grouped_data = filtered_df.groupby('Month', as_index=False).agg({
    'QTY': 'sum'
})

# Create a bar plot using plotly express for the product count by month
fig = px.bar(
    grouped_data,
    x='Month',
    y='QTY',
    color='Month',
    title='Distribution of Product Count by Month',
    labels={'Month': 'Month', 'QTY': 'Product Count (Quantity Sold)'},
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)


example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product count by Month", text=example_plotly)
# fig.show()


/tmp/ipython-input-2688429557.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-2688429557.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@144ea9ec-1046-4253-8b68-ee0ef6eae0f5@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eProduct Count (Quantity Sold)=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@January@@],@@xaxis@@:@@x@@,@@y@@:[11544],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eProduct Count (Quantity Sold)=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@February@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@February@@,@@offsetgroup@@:@@February@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textpo

In [158]:
progress = "50"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "50", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:07 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df6e8b50>: Failed to establish a new connection: [Errno 111] Connection refused'))


### Using `SKUID`

In [159]:
# Convert the invoice date column to datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month, summing the quantity (QTY) sold
grouped_data = filtered_df.groupby('Month', as_index=False).agg({
    'SKUID': 'count'
})

# Create a bar plot using plotly express for the product count by month
fig = px.bar(
    grouped_data,
    x='Month',
    y='SKUID',
    color='Month',
    title='Distribution of Product Count by Month',
    labels={'Month': 'Month', 'SKUID': 'Product Count'},
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product count by Month", text=example_plotly)


# fig.show()


/tmp/ipython-input-3101653313.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-3101653313.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@5440d6f1-ff3d-4e30-a4a2-d9ff7af4f253@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eProduct Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@January@@],@@xaxis@@:@@x@@,@@y@@:[330],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=%{x}\\u003cbr\\u003eProduct Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@February@@,@@marker@@:{@@color@@:@@#FFE3B2@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@February@@,@@offsetgroup@@:@@February@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Februar

In [160]:
progress = "55"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "55", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:07 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65dfb72710>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Invoices by Month for each customer

In [161]:
# Convert the invoice date column to datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Group the data by month and customer, and count the number of invoices
grouped_data = filtered_df.groupby(['Month', 'CUST_NAME'], as_index=False).agg({
    'INV_NO': 'count'
})

# Create a bar plot using plotly express for invoice count by month and customer
fig = px.bar(
    grouped_data,
    x='CUST_NAME',
    y='INV_NO',
    color='Month',
    title='Distribution of Invoices by Month for Each Customer',
    labels={'CUST_NAME': 'Customer', 'INV_NO': 'Number of Invoices', 'Month': 'Month'},
    barmode='stack',
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Invoices by Month for each customer", text=example_plotly)


# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@dd98f755-ab09-4ad0-929d-8054a6e22878@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=January\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eNumber of Invoices=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Ballard Rowland@@,@@English Ruiz@@,@@Kane Robertson@@,@@Norman and Collins Carr@@,@@Patterson and Strickland Downs@@,@@Villa and Ray Brown@@,@@White Johnson@@],@@xaxis@@:@@x@@,@@y@@:[30,30,30,1,31,30,8],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=February\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eNumber of Invoices=%{y}\\u003cextra\\u003e\\u003c\\u002fextr

/tmp/ipython-input-3695871730.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-3695871730.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [162]:
progress = "60"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "60", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:07 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65dfb2ab10>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Net sales amount by Month for each customer
This graph depicts Customer wise net sales done in each month.

In [163]:
# Ensure the invoice date column is in datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month and customer, summing the net sales (NET column)
grouped_data = filtered_df.groupby(['Month', 'CUST_NAME'], as_index=False).agg({
    'NET': 'sum'
})

# Create a stacked bar plot using plotly express for net sales by month and customer
fig = px.bar(
    grouped_data,
    x='CUST_NAME',
    y='NET',
    color='Month',
    title='Distribution of Net Sales Amount by Month for each customer',
    labels={'CUST_NAME': 'Customer', 'NET': 'Net Sales Amount', 'Month': 'Month'},
    barmode='stack',  # Setting barmode to 'stack' for stacked bars
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Net sales amount by Month for each customer", text=example_plotly)


# fig.show()


/tmp/ipython-input-2311307371.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-2311307371.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@52aa3a16-d62a-4504-8282-cc9a43487fec@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=January\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eNet Sales Amount=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Ballard Rowland@@,@@English Ruiz@@,@@Kane Robertson@@,@@Norman and Collins Carr@@,@@Patterson and Strickland Downs@@,@@Villa and Ray Brown@@,@@White Johnson@@],@@xaxis@@:@@x@@,@@y@@:[17.88634855,120.8671015,120.8671015,3.75547,18.55104685,17.88634855,27.87201],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=February\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eNet 

In [164]:
progress = "65"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "65", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:07 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df5bdc90>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Distribution of Product count by Month for each customer
This graph depicts Customer wise product count sold in each month.

### Using `QTY`

In [165]:
# Ensure the invoice date column is in datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month and customer, summing the quantity (QTY)
grouped_data = filtered_df.groupby(['Month', 'CUST_NAME'], as_index=False).agg({
    'QTY': 'sum'
})

# Create a stacked bar plot using plotly express for product count by month and customer
fig = px.bar(
    grouped_data,
    x='CUST_NAME',
    y='QTY',
    color='Month',
    title='Distribution of Product Count by Month for Each Customer',
    labels={'CUST_NAME': 'Customer', 'QTY': 'Product Count (Quantity Sold)', 'Month': 'Month'},
    barmode='stack',  # Setting barmode to 'stack' for stacked bars
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product count by Month for each customer", text=example_plotly)


# fig.show()


/tmp/ipython-input-1250227174.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-1250227174.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@9053ecaa-ab1e-45b8-8546-8589d96f8598@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=January\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eProduct Count (Quantity Sold)=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Ballard Rowland@@,@@English Ruiz@@,@@Kane Robertson@@,@@Norman and Collins Carr@@,@@Patterson and Strickland Downs@@,@@Villa and Ray Brown@@,@@White Johnson@@],@@xaxis@@:@@x@@,@@y@@:[1150,992,992,23,1183,1150,237],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=February\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eProduct Count (Quantity Sold)=%{y}\\u0

In [166]:
progress = "70"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "70", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:07 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e16d6990>: Failed to establish a new connection: [Errno 111] Connection refused'))


### Using `SKUID`

In [167]:
# Ensure the invoice date column is in datetime format
filtered_df['INV_DT'] = pd.to_datetime(filtered_df['INV_DT'], errors='coerce')

# Extract the month name from the invoice date
filtered_df['Month'] = filtered_df['INV_DT'].dt.strftime('%B')

# Grouping the data by month and customer, summing the quantity (QTY)
grouped_data = filtered_df.groupby(['Month', 'CUST_NAME'], as_index=False).agg({
    'SKUID': 'count'
})

# Create a stacked bar plot using plotly express for product count by month and customer
fig = px.bar(
    grouped_data,
    x='CUST_NAME',
    y='SKUID',
    color='Month',
    title='Distribution of Product Count by Month for Each Customer',
    labels={'CUST_NAME': 'Customer', 'SKUID': 'Product Count', 'Month': 'Month'},
    barmode='stack',  # Setting barmode to 'stack' for stacked bars
    category_orders={'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']},
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)


test = plotly.offline.plot(fig, output_type='div', include_plotlyjs=False)
example_plotly = f'{test}'
restworkflowcontext.outPlotly(9, title="Distribution of Product count by Month for each customer", text=example_plotly)


# fig.show()


Check the Input type..
There is a match
 And Input is string type.
Sending Message: {'text': 'Plotly.newPlot(                        @@c2c1b635-8401-48df-9d23-e2df39251132@@,                        [{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=January\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eProduct Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@,@@legendgroup@@:@@January@@,@@marker@@:{@@color@@:@@#77C8D0@@,@@pattern@@:{@@shape@@:@@@@}},@@name@@:@@January@@,@@offsetgroup@@:@@January@@,@@orientation@@:@@v@@,@@showlegend@@:true,@@textposition@@:@@auto@@,@@x@@:[@@Ballard Rowland@@,@@English Ruiz@@,@@Kane Robertson@@,@@Norman and Collins Carr@@,@@Patterson and Strickland Downs@@,@@Villa and Ray Brown@@,@@White Johnson@@],@@xaxis@@:@@x@@,@@y@@:[30,30,30,1,31,30,8],@@yaxis@@:@@y@@,@@type@@:@@bar@@},{@@alignmentgroup@@:@@True@@,@@hovertemplate@@:@@Month=February\\u003cbr\\u003eCustomer=%{x}\\u003cbr\\u003eProduct Count=%{y}\\u003cextra\\u003e\\u003c\\u002fextra\\u003e@@

/tmp/ipython-input-3173323758.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-3173323758.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [168]:
progress = "75"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "75", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e16376d0>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Summary

In [169]:
# print(filtered_df.describe())
restworkflowcontext.outPandasDataframe(9,"filtered_df.describe()", filtered_df.describe())

Sending Message: {"cellValues": [["INV_DT", "QTY", "GROSS", "UNITPRICE", "NET_TOT_TAX", "ADJ_AMT", "CUST_DISC_AMT", "PRODUCT_DISC_AMT", "NET"], ["object", "float64", "float64", "float64", "float64", "float64", "float64", "float64", "float64"], [np.int64(2782), 2782.0, 2782.0, 2782.0, 2782.0, 2782.0, 2782.0, 2782.0, 2782.0], [Timestamp("2023-05-16 21:18:30.280373760"), 36.89827462257369, 2.6215175758447162, 0.07507450291265276, 10.0, 0.0, 4.0, 1.0, 2.2282899394680085], [Timestamp("2023-01-02 00:00:00"), 4.0, 0.044, 0.010628571, 10.0, 0.0, 4.0, 1.0, 0.0374], [Timestamp("2023-02-22 00:00:00"), 25.0, 0.6964, 0.01950589325, 10.0, 0.0, 4.0, 1.0, 0.59194], [Timestamp("2023-05-09 00:00:00"), 34.0, 1.556798, 0.06609671, 10.0, 0.0, 4.0, 1.0, 1.3232783], [Timestamp("2023-07-30 00:00:00"), 46.0, 4.26894625, 0.11924972525, 10.0, 0.0, 4.0, 1.0, 3.6286043125], [Timestamp("2023-11-30 00:00:00"), 114.0, 12.878397, 0.26395996, 10.0, 0.0, 4.0, 1.0, 10.94663745], [nan, 16.81094200822638, 2.358350237646524

In [170]:
progress = "80"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "80", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df863610>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Null Values in Columns

In [171]:
# Get the number of null values for each column
null_count = filtered_df.isnull().sum()
null_percentage = (filtered_df.isnull().sum() / len(filtered_df)) * 100

null_data = pd.DataFrame({
    'Null Count': null_count,
    'Null Percentage': null_percentage
})

null_data.reset_index(inplace=True)
null_data.columns = ['Column', 'Number of Null Values', 'Percentage of Null Values']

restworkflowcontext.outPandasDataframe(9,"null_data", null_data)
# null_data

Sending Message: {"cellValues": [["Column", "Number of Null Values", "Percentage of Null Values"], ["object", "int64", "float64"], ["INV_DT", 0, 0.0], ["INV_NO", 0, 0.0], ["INV_NO_KEY", 0, 0.0], ["QTY", 0, 0.0], ["GROSS", 0, 0.0], ["UNITPRICE", 0, 0.0], ["NET_TOT_TAX", 0, 0.0], ["ADJ_AMT", 0, 0.0], ["CUST_DISC_AMT", 0, 0.0], ["PRODUCT_DISC_AMT", 0, 0.0]], "id": 9, "name": "Pandas DataFrame", "title": "null_data", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df6d2250>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Columns Unique Count

In [172]:
columns_to_check = ['CUSTOMER_ID', 'CUST_NAME', 'CATEGORY', 'SUBCATEGPRY',
                    'SKUID', 'COUNTRY_ID', 'SECTOR', 'DISTRIBUTOR_ID', 'DIST_NAME']

unique_values = filtered_df[columns_to_check].nunique()

unique_values_df = pd.DataFrame({
    'Column': unique_values.index,
    'Unique Values Count': unique_values.values
})

restworkflowcontext.outPandasDataframe(9,"unique_values_df", unique_values_df)
# unique_values_df


Sending Message: {"cellValues": [["Column", "Unique Values Count"], ["object", "int64"], ["CUSTOMER_ID", 27], ["CUST_NAME", 8], ["CATEGORY", 2], ["SUBCATEGPRY", 2], ["SKUID", 2], ["COUNTRY_ID", 1], ["SECTOR", 1], ["DISTRIBUTOR_ID", 1], ["DIST_NAME", 1]], "id": 9, "name": "Pandas DataFrame", "title": "unique_values_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df7f2c90>: Failed to establish a new connection: [Errno 111] Connection refused'))


## CUST_NAME

In [173]:
cust_name_count_df = filtered_df['CUST_NAME'].value_counts().reset_index()
restworkflowcontext.outPandasDataframe(9,"cust_name_count_df", cust_name_count_df)
# cust_name_count_df

Sending Message: {"cellValues": [["CUST_NAME", "count"], ["object", "int64"], ["Patterson and Strickland Downs", 394], ["Villa and Ray Brown", 363], ["Ballard Rowland", 211], ["English Ruiz", 148], ["Kane Robertson", 148], ["White Johnson", 64], ["Martinez PLC", 61], ["Norman and Collins Carr", 3]], "id": 9, "name": "Pandas DataFrame", "title": "cust_name_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df9642d0>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [174]:
progress = "85"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "85", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df81ec90>: Failed to establish a new connection: [Errno 111] Connection refused'))


## CUSTOMER_ID

In [175]:
cust_id_count_df = filtered_df['CUSTOMER_ID'].value_counts().reset_index()
restworkflowcontext.outPandasDataframe(9,"cust_id_count_df", cust_id_count_df)
# cust_id_count_df

Sending Message: {"cellValues": [["CUSTOMER_ID", "count"], ["object", "int64"], ["CUST10092", 394], ["CUST10047", 363], ["CUST10077", 211], ["CUST10032", 152], ["CUST10062", 152], ["CUST10046", 148], ["CUST10031", 148], ["CUST10061", 148], ["CUST10091", 148], ["CUST10076", 148]], "id": 9, "name": "Pandas DataFrame", "title": "cust_id_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e05f9090>: Failed to establish a new connection: [Errno 111] Connection refused'))


## SECTOR

In [176]:
sector_count_df = filtered_df['SECTOR'].value_counts().reset_index()
restworkflowcontext.outPandasDataframe(9,"sector_count_df", sector_count_df)
# sector_count_df

Sending Message: {"cellValues": [["SECTOR", "count"], ["object", "int64"], ["BCC", 2782]], "id": 9, "name": "Pandas DataFrame", "title": "sector_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df7f2fd0>: Failed to establish a new connection: [Errno 111] Connection refused'))


## CATEGORY

In [177]:
category_count_df = filtered_df['CATEGORY'].value_counts().reset_index()
restworkflowcontext.outPandasDataframe(9,"category_count_df", category_count_df)
# category_count_df

Sending Message: {"cellValues": [["CATEGORY", "count"], ["object", "int64"], ["WIPES", 1524], ["LOTIONS", 1258]], "id": 9, "name": "Pandas DataFrame", "title": "category_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df6d2250>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [178]:
progress = "90"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "90", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df81e9d0>: Failed to establish a new connection: [Errno 111] Connection refused'))


## SUBCATEGORY

In [179]:
subcategory_count_df = filtered_df['SUBCATEGPRY'].value_counts().reset_index()
restworkflowcontext.outPandasDataframe(9,"subcategory_count_df", subcategory_count_df)
subcategory_count_df

Sending Message: {"cellValues": [["SUBCATEGPRY", "count"], ["object", "int64"], ["Huggies", 1524], ["Pampers", 1258]], "id": 9, "name": "Pandas DataFrame", "title": "subcategory_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65e2dcabd0>: Failed to establish a new connection: [Errno 111] Connection refused'))


,SUBCATEGPRY,count
0,Huggies,1524
1,Pampers,1258


In [180]:
def get_counts(filtered_df, col):
    count_df = filtered_df[col].value_counts().reset_index()
    restworkflowcontext.outPandasDataframe(9,"category_count_df", count_df)
    count_df

In [181]:
get_counts(filtered_df, ['DISTRIBUTOR_ID'])

Sending Message: {"cellValues": [["DISTRIBUTOR_ID", "count"], ["object", "int64"], ["DT3501", 2782]], "id": 9, "name": "Pandas DataFrame", "title": "category_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df967b90>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [182]:
progress = "95"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "95", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df004e90>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [183]:
get_counts(filtered_df, ['SKUID'])

Sending Message: {"cellValues": [["SKUID", "count"], ["object", "int64"], ["GZ0G57", 1524], ["M4N5O6", 1258]], "id": 9, "name": "Pandas DataFrame", "title": "category_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df6fa750>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [184]:
get_counts(filtered_df, ['DIST_NAME'])

Sending Message: {"cellValues": [["DIST_NAME", "count"], ["object", "int64"], ["Global Enterprises", 2782]], "id": 9, "name": "Pandas DataFrame", "title": "category_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df81f910>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [185]:
get_counts(filtered_df, ['COUNTRY_ID'])

Sending Message: {"cellValues": [["COUNTRY_ID", "count"], ["object", "int64"], ["US", 1392]], "id": 9, "name": "Pandas DataFrame", "title": "category_count_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df6d2bd0>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Skewness and Kurtosis

In [186]:
columns_to_check = ['QTY', 'GROSS', 'UNITPRICE', 'NET']

skewness_values = filtered_df[columns_to_check].skew()
kurtosis_values = filtered_df[columns_to_check].kurt()

stats_df = pd.DataFrame({
    'Skewness': skewness_values,
    'Kurtosis': kurtosis_values
})

restworkflowcontext.outPandasDataframe(9,"stats_df", stats_df)

# stats_df

Sending Message: {"cellValues": [["Skewness", "Kurtosis"], ["float64", "float64"], [1.100856865635047, 2.053341329671536], [1.0552902121271797, 0.5348984940443264], [0.7366706032042404, -0.4766266132314976], [1.05529021212718, 0.5348984940443278]], "id": 9, "name": "Pandas DataFrame", "title": "stats_df", "type": "table", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df701690>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [187]:
progress = "100"
restworkflowcontext.outputProgress(9, title="Progress", progress=progress)

Sending Message: {"progress": "100", "id": 9, "name": "Progress", "title": "Progress", "type": "progress", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df545ed0>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [188]:
message = "Sending the success message."
restworkflowcontext.outSuccess(9, title="Success", text=message)

Sending Message: {"text": "Sending the success message.", "id": 9, "name": "Success", "title": "Success", "type": "success", "resultType": 3, "visibility": "EXPANDED", "time": "Aug 18, 2025 07:23:08 AM"}
Error : HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c65df7f2fd0>: Failed to establish a new connection: [Errno 111] Connection refused'))
